# <center> <font color='#0B5345'> <b> Predefined callbacks </font> </b> </center>

### <b> <font color='blue'> Table of Contents </b> </font>

- 1. [Libraries](#1)
- 2. [Loading data](#2)
- 3. [Model Building](#3)
- 4. [Training with callbacks](#4)
   - 4.1. [Early Stopping](#4.1)
   - 4.2. [Learning Rate Scheduler](#4.2)
   - 4.3. [Tensor Board](#4.3)
   - 4.4. [Checkpoint](#4.4)
   - 4.5. [Combining callbacks](#4.5)





<a name="1"></a>
## <b> <font color='##138D75'> 1. Libraries </b> </font>

In [21]:
# que no se impriman info y warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import warnings
# Ignore specific warning
warnings.filterwarnings('ignore', message='Allocation of .* exceeds 10% of free system memory')


In [59]:
import tensorflow as tf
from tensorflow.keras import layers, callbacks, models
from tensorflow.keras.models import load_model
import datetime


<a name="2"></a>
## <b> <font color='##138D75'> 2. Loading data </b> </font>

In [14]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist

# Load data into training and testing sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0


<a name="2"></a>
## <b> <font color='##138D75'> 3. Model building </b> </font>

Let's create a simple model (our goal here is to learn about callbacks)

In [56]:
def build_model():

    # Define the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.keras.layers.Dense(10),
        tf.keras.layers.Dense(10,activation='softmax'),
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
        
    return model

<a name="4"></a>
## <b> <font color='##138D75'> 4. Training with Early Stopping callback </b> </font>

<a name="4.1"></a>
### 4.1. Early Stopping

The "Early Stopping Callback" stops training when a monitored metric has stopped improving.

It is a way to prevent overfitting.

First, let's use the "bad model"

In [57]:
model = build_model()

In [23]:
# Define EarlyStopping callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    min_delta=0.001, 
    restore_best_weights=True
)

# Train the model with EarlyStopping callback
history = model.fit(x_train, y_train, epochs=20, 
                    validation_data=(x_test, y_test), 
                    callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4464 - accuracy: 0.8782 - val_loss: 0.2987 - val_accuracy: 0.9175
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2941 - accuracy: 0.9168 - val_loss: 0.2785 - val_accuracy: 0.9202
Epoch 3/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2786 - accuracy: 0.9218 - val_loss: 0.2739 - val_accuracy: 0.9241
Epoch 4/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2708 - accuracy: 0.9254 - val_loss: 0.2771 - val_accuracy: 0.9252
Epoch 5/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2653 - accuracy: 0.9263 - val_loss: 0.2748 - val_accuracy: 0.9237
Epoch 6/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2612 - accuracy: 0.9277 - val_loss: 0.2725 - val_accuracy: 0.9235
Epoch 7/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2583 - accuracy: 0.9288 - val_loss: 0.2783 - val_accuracy:

We can see that it stops in the epoch 13 because in 3 consecutives epochs the validation accuracy did not improve (0.2735 -> 0.2826 -> 0.2763)

<a name="4.2"><a/>
### Learning Rate Scheduler

The Learning Rate Scheduler callback in TensorFlow allows you to dynamically adjust the learning rate during training based on predefined schedules or functions. It helps optimize the training process by allowing the model to converge faster and potentially achieve better performance.

In [58]:
model = build_model()

In [35]:
# Define a simple learning rate schedule function
#In this example, we use a simple decay schedule where 
#the learning rate decreases by a factor of 0.1 every 5 epochs.
def lr_schedule(epoch):
    """
    Returns a learning rate based on the current epoch.
    """
    initial_lr = 0.1
    decay_factor = 0.1
    decay_epochs = 5
    lr = initial_lr * (decay_factor ** (epoch // decay_epochs))
    return lr

# Create a LearningRateScheduler callback
lr_scheduler = callbacks.LearningRateScheduler(lr_schedule)


# Train the model with the learning rate scheduler callback
history = model.fit(x_train, y_train, epochs=12,
                    validation_data=(x_test, y_test), 
                    callbacks=[lr_scheduler])


Epoch 1/12
1875/1875 [==============================] - 3s 1ms/step - loss: 2.7012 - accuracy: 0.7847 - val_loss: 2.9506 - val_accuracy: 0.8246 - lr: 0.1000
Epoch 2/12
1875/1875 [==============================] - 2s 1ms/step - loss: 3.1615 - accuracy: 0.8035 - val_loss: 2.9053 - val_accuracy: 0.8268 - lr: 0.1000
Epoch 3/12
1875/1875 [==============================] - 2s 1ms/step - loss: 3.1910 - accuracy: 0.8090 - val_loss: 4.0559 - val_accuracy: 0.8019 - lr: 0.1000
Epoch 4/12
1875/1875 [==============================] - 2s 1ms/step - loss: 3.0937 - accuracy: 0.8156 - val_loss: 4.4215 - val_accuracy: 0.7731 - lr: 0.1000
Epoch 5/12
1875/1875 [==============================] - 2s 1ms/step - loss: 3.3387 - accuracy: 0.8122 - val_loss: 3.5317 - val_accuracy: 0.8326 - lr: 0.1000
Epoch 6/12
1875/1875 [==============================] - 2s 1ms/step - loss: 0.8352 - accuracy: 0.8987 - val_loss: 0.5338 - val_accuracy: 0.9024 - lr: 0.0100
Epoch 7/12
1875/1875 [==============================] - 2s

We can see that when it reaches epoch 6 (after 5 epochs), the learning rate changes from 0.1 to 0.01, and when it reaches epoch 11 (after another 5 epochs), from 0.01 to 0.001."

<a name="4.3"></a>
### Tensor Board callback

The TensorBoard callback in TensorFlow is used to visualize and monitor the training process of your neural network models.

In [37]:
model = build_model()

# Define TensorBoard callback
# We specify yhe log directory where the TensorBoard logs will be saved. The log directory includes 
# a timestamp to make it unique.
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
tensorboard_callback = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model with TensorBoard callback
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])


Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4565 - accuracy: 0.8756 - val_loss: 0.2929 - val_accuracy: 0.9176
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2928 - accuracy: 0.9176 - val_loss: 0.2782 - val_accuracy: 0.9188
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2774 - accuracy: 0.9225 - val_loss: 0.2680 - val_accuracy: 0.9245
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2691 - accuracy: 0.9252 - val_loss: 0.2724 - val_accuracy: 0.9257
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2649 - accuracy: 0.9268 - val_loss: 0.2685 - val_accuracy: 0.9252
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2608 - accuracy: 0.9281 - val_loss: 0.2713 - val_accuracy: 0.9258
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2572 - accuracy: 0.9288 - val_loss: 0.2829 - val_accuracy:


After training, you can start TensorBoard from the command line by navigating to the directory containing your code and logs:

.....Predefined$ ls


'Callbacks.ipnyb' logs

Start Tensor Board

...... Predefined$ tensorboard --logdir logs/


Then, we navigate to 'http://localhost:6006'


<br>

<img src="images/TensorBoard.png"/>

<a name="4.4"></a>
### Checkpoint 

The Checkpoint callback allows you to save the model's weights during training, which enables you to resume training from the last saved checkpoint or use the saved weights for inference later on. This callback is particularly useful in scenarios where you have long training times or when you want to track the progress of your model over multiple training sessions


We are going to simulate a scenario where the training is interrupted (for example, if the computer crashes) to demonstrate how we can resume training not from the beginning, but from the last saved checkpoint.

In [42]:
model = build_model()

# Define checkpoint callback to save model weights
checkpoint_path = "training/model_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
checkpoint_callback = callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                      monitor='val_loss',
                                      save_best_only=True, # only saves the best
                                      mode='min') # it's min because monitor=val_loss



# Train the model with checkpoint callback
model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test),
          callbacks=[checkpoint_callback])

Epoch 1/20
1874/1875 [============================>.] - ETA: 0s - loss: 0.2508 - accuracy: 0.9310INFO:tensorflow:Assets written to: training/model_checkpoint.ckpt/assets


INFO:tensorflow:Assets written to: training/model_checkpoint.ckpt/assets


1875/1875 [==============================] - 3s 1ms/step - loss: 0.2507 - accuracy: 0.9310 - val_loss: 0.2721 - val_accuracy: 0.9251
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2498 - accuracy: 0.9305 - val_loss: 0.2735 - val_accuracy: 0.9249
Epoch 3/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2485 - accuracy: 0.9315 - val_loss: 0.2814 - val_accuracy: 0.9225
Epoch 4/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2471 - accuracy: 0.9312 - val_loss: 0.2729 - val_accuracy: 0.9257
Epoch 5/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2465 - accuracy: 0.9320 - val_loss: 0.2725 - val_accuracy: 0.9267
Epoch 6/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2460 - accuracy: 0.9313 - val_loss: 0.2753 - val_accuracy: 0.9254
Epoch 7/20
1353/1875 [====================>.........] - ETA: 0s - loss: 0.2417 - accuracy: 0.9334

KeyboardInterrupt: 

In [46]:
# Load the model
model_ckp = load_model(checkpoint_path)

In [48]:
# check
model_ckp.evaluate(x_test,y_test)

313/313 [==============================] - 0s 953us/step - loss: 0.2721 - accuracy: 0.9251


[0.27205604314804077, 0.9251000285148621]

In [49]:
# resume training
model_ckp.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test),
          callbacks=[checkpoint_callback])

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2488 - accuracy: 0.9312 - val_loss: 0.2724 - val_accuracy: 0.9235
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2484 - accuracy: 0.9314 - val_loss: 0.2722 - val_accuracy: 0.9262
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2481 - accuracy: 0.9315 - val_loss: 0.2763 - val_accuracy: 0.9264
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2467 - accuracy: 0.9313 - val_loss: 0.2763 - val_accuracy: 0.9263
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2460 - accuracy: 0.9311 - val_loss: 0.2759 - val_accuracy: 0.9257


<a name="4.5"></a>
### Combining callbacks

We can combine callbacks. In the following example, we combine 'Early Stopping' with 'Learning Rate Scheduler'


In [54]:
model = build_model()

In [55]:
# Define EarlyStopping callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    min_delta=0.001, 
    restore_best_weights=True
)


# LR Scheduler
# This function keeps the initial learning rate for the first 5 epochs
# and decreases it exponentially after that.
def scheduler(epoch, lr):
     if epoch < 3:
        return lr
     else:
        return lr * ops.exp(-0.1)


    
lr_schduler = callbacks.LearningRateScheduler(scheduler)

    
# Train the model with EarlyStopping callback
history = model.fit(x_train, y_train, epochs=20, 
                    validation_data=(x_test, y_test), 
                    callbacks=[early_stopping, lr_scheduler])

Epoch 1/20
1875/1875 [==============================] - 3s 1ms/step - loss: 3.0866 - accuracy: 0.7808 - val_loss: 3.2225 - val_accuracy: 0.8380 - lr: 0.1000
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 3.3037 - accuracy: 0.8058 - val_loss: 2.0044 - val_accuracy: 0.8497 - lr: 0.1000
Epoch 3/20
1875/1875 [==============================] - 2s 1ms/step - loss: 2.9575 - accuracy: 0.8112 - val_loss: 3.7519 - val_accuracy: 0.7789 - lr: 0.1000
Epoch 4/20
1875/1875 [==============================] - 2s 1ms/step - loss: 3.2227 - accuracy: 0.8120 - val_loss: 3.8988 - val_accuracy: 0.8007 - lr: 0.1000
Epoch 5/20
1875/1875 [==============================] - 3s 1ms/step - loss: 3.3476 - accuracy: 0.8130 - val_loss: 3.9876 - val_accuracy: 0.8066 - lr: 0.1000
Epoch 6/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.7884 - accuracy: 0.8975 - val_loss: 0.5049 - val_accuracy: 0.9004 - lr: 0.0100
Epoch 7/20
1875/1875 [==============================] - 2s